## Recipe Builder Actions Overview

### Saving a File Cell
If you wish to save the contents of a cell, simply run it.  The `%%writefile` command at the top of the cell will write the contents of the cell to the file named at the top of the cell. You should run the cells manually when applicable. However, **pressing any of the actions at the top will automatically run all file cells relevant to the action**.

### Training and Scoring
Press the associated buttons at the top in order to run training or scoring. The training output will be shown below the `evaluator.py` cell and scoring output will be shown below the `datasaver.py` cell. You must run training at least once before you can run scoring. You may delete the output cell(s). Running training the first time or after changing `requirements.txt` will be slower since the dependencies for the recipe need to be installed, but subsequent runs will be significantly faster. If you wish to see the hidden output add `debug` to the end of the output cell and re-run it.

### Creating the Recipe
When you are done editing the recipe and satisfied with the training/scoring output, you can create a recipe from the notebook by pressing `Create Recipe`. You must run scoring at least once before you can create the recipe. After pressing it, you will see a progress bar showing how much time is left for the build to finish. If the recipe creation is successful the progress bar will be replaced by an external link that you can click to navigate to the created recipe.


## Caution!
* **Do not delete any of the file cells**
* **Do not edit the `%%writefile` line at the top of the file cells**

---

#### **Requirements File** (Optional)
Add additional libraries you wish to use in the recipe to the cell below. You can specify the version number if necessary. The file cell below is a **commented out example**.

In [ ]:
# pandas=0.22.0
# numpy

Search here for additional libraries https://anaconda.org/. This is the list of main **libraries already in use**:
`python=3.5.2` `scikit-learn` `pandas` `numpy` `data_access_sdk_python`
**Warning: libraries or specific versions you add may be incompatible with the above libraries**.

---

#### **Configuration Files**
List any hyperparamters you wish to use. Specify the dataset(s) and schema(s) that are needed for training/scoring. To find the dataset ids go to the **Data tab** in Adobe Experience Platform or view the **Datasets** folder in the **Notebooks Data tab** on the left. You can also find schema id in the **Notebooks Data tab** under the **Schemas** folder. Each configuration will only be used for its corresponding action. `ACP_DSW_TRAINING_XDM_SCHEMA` and `ACP_DSW_SCORING_RESULTS_XDM_SCHEMA` will only be used after the recipe has been created.

##### Training Configuration

In [ ]:
{
   "trainingDataSetId": "<replace with training dataset id>",
   "ACP_DSW_TRAINING_XDM_SCHEMA": "<replace with training xdm schema id>",
   "num_recommendations": "5",
   "sampling_fraction": "0.5",
   "tenant_id":"<replace with tenant id>" 
}

##### Scoring Configuration

In [ ]:
{
   "scoringDataSetId": "<replace with training dataset id>",
   "scoringResultsDataSetId": "<replace with scoring results dataset id>",
   "ACP_DSW_SCORING_RESULTS_XDM_SCHEMA": "<replace with scoring xdm schema id>"
}

**The following configuration parameters are automatically set for you when you train/score:** 
`ML_FRAMEWORK_IMS_USER_CLIENT_ID` `ML_FRAMEWORK_IMS_TOKEN` `ML_FRAMEWORK_IMS_ML_TOKEN` `ML_FRAMEWORK_IMS_TENANT_ID` `saveData`

---

#### **Training Data Loader File**
Implement the `load` function to load and prepare the training data.

In [ ]:
import numpy as np
import pandas as pd
from platform_sdk.dataset_reader import DatasetReader
from .utils import get_client_context

def load(configProperties):
    print("Training Data Load Start")
    print(configProperties)
    client_context = get_client_context(configProperties)

    dataset_reader = DatasetReader(client_context, configProperties['trainingDataSetId'])
    df = dataset_reader.read()
    df.head()

    print("Training Data Load Finish")
    return df


---

#### **Scoring Data Loader File**
Implement the `load` function to load and prepare the scoring data.

In [ ]:
import numpy as np
import pandas as pd
from platform_sdk.dataset_reader import DatasetReader
from .utils import get_client_context

def load(configProperties):

    print("Scoring Data Load Start")

    #########################################
    # Load Data
    #########################################
    client_context = get_client_context(configProperties)
    dataset_reader = DatasetReader(client_context, configProperties['scoringDataSetId'])
    df = dataset_reader.read()

    print("Scoring Data Load Finish")

    return df


---

#### **Pipeline File**
Implement the `train` function and return the trained model. Implement the `score` function to return a prediction made on the scoring data.

In [ ]:
import pandas as pd
import numpy as np
import random
from collections import Counter

class PopularityBasedRecommendationModel():
    def __init__(self, num_to_recommend, configProperties):
        self.num_to_recommend = num_to_recommend
        self.recommendations = ['dummy']

        tenant_id = configProperties.get('tenant_id')
        
        sf = 'sampling_fraction'
        if sf in configProperties:
            cpsf = float(configProperties[sf])
            sampling_fraction =  cpsf if (cpsf >= 0 and cpsf <= 1) else 0.5
        else:
            sampling_fraction = 0.5
        
        print("sf", sampling_fraction)

        self.sampling_fraction = sampling_fraction
        self.user_id_column = '_%s.userId' % tenant_id
        self.recommendations_column = '_%s.recommendations' % tenant_id
        self.item_id_column = '_%s.itemId' % tenant_id

    def fit(self, df):
        df = df[df[self.item_id_column].notnull()]
        all_values = list(df[self.item_id_column].values)
        total_cnt = len(all_values)
        sample_size = int(round(total_cnt * self.sampling_fraction))
        sampled_values = all_values[0:sample_size]
        print("sv", len(sampled_values))
        self.recommendations = [item for item, freq in
                                Counter(sampled_values).most_common(self.num_to_recommend)]


    def predict(self, df):
        # remove columns having none
        df = df[df[self.item_id_column].notnull()]

        df_grouped_by_user = df.groupby(self.user_id_column).agg(
            {self.item_id_column: lambda x: ','.join(x)})\
        .rename(columns={self.item_id_column:'interactions'}).reset_index()

        df_grouped_by_user[self.recommendations_column] = '#'.join(self.recommendations)
        df_grouped_by_user = df_grouped_by_user.drop(['interactions'],axis=1)

        return df_grouped_by_user

def train(configProperties, data):

    print("Train Start")

    #########################################
    # Extract fields from configProperties
    #########################################
    num_recommendations = int(configProperties['num_recommendations'])

    #########################################
    # Fit model
    #########################################
    print('in train')
    print(configProperties)

    model = PopularityBasedRecommendationModel(num_recommendations, configProperties)

    model.fit(data)

    print("Train Complete")

    return model

def score(configProperties, data, model):

    print("Score Start")

    result = model.predict(data)

    print("Score Complete")

    return result


---

#### **Evaluator File**
Implement the `split` function to partition the training data and the `evaluate` function to the return the validation metrics you wish to see. Training output will be shown below this file cell.

In [ ]:
from ml.runtime.python.Interfaces.AbstractEvaluator import AbstractEvaluator
import numpy as np
import pandas as pd

class Evaluator(AbstractEvaluator):

    def split(self, configProperties={}, dataframe=None):
        #########################################
        # Load Data
        #########################################
        train = dataframe[:]
        test = dataframe[:]

        return train, test

    def evaluate(self, data=[], model={}, configProperties={}):
        print ("Evaluation evaluate triggered")

        tenant_id = configProperties.get('tenant_id')

        self.user_id_column = '_%s.userId' % tenant_id
        self.recommendations_column = '_%s.recommendations' % tenant_id
        self.item_id_column = '_%s.itemId' % tenant_id

        # remove columns having none
        data = data[data[self.item_id_column].notnull()]

        data_grouped_by_user = data.groupby(self.user_id_column).agg(
            {self.item_id_column: lambda x: '#'.join(x)})\
        .rename(columns={self.item_id_column:'interactions'}).reset_index()

        data_recommendations = model.predict(data)

        merged_df = pd.merge(data_grouped_by_user, data_recommendations, on=[self.user_id_column]).reset_index()

        def compute_recall(row):
            set_interactions = set(row['interactions'].split('#'))
            set_recommendations = set(row[self.recommendations_column].split('#'))
            inters = set_interactions.intersection(set_recommendations)
            if len(inters) > 0:
                return 1
            return 0

        def compute_precision(row):
            set_interactions = set(row['interactions'].split('#'))
            list_recommendations = row[self.recommendations_column].split('#')
            score = 0
            weight = 0.5
            for rec in list_recommendations:
                if rec in set_interactions:
                    score = score + weight
                weight = weight / 2

            return score


        merged_df['recall'] = merged_df.apply(lambda row: compute_recall(row), axis=1)
        merged_df['precision'] = merged_df.apply(lambda row: compute_precision(row), axis=1)

        recall = merged_df['recall'].mean()
        precision = merged_df['precision'].mean()

        metric = [{"name": "Recall", "value": recall, "valueType": "double"},
                 {"name": "Precision", "value": precision, "valueType": "double"}]

        print(metric)

        return metric
    

---

#### **Data Saver File**
Implement the `save` function for saving your prediction. Scoring output will be added below this cell.

In [ ]:
from platform_sdk.models import Dataset
from platform_sdk.dataset_writer import DatasetWriter
from .utils import get_client_context
from functools import reduce
import json

def save(configProperties, prediction):

    print(prediction)
    client_context = get_client_context(configProperties)
    dataset = Dataset(client_context).get_by_id(configProperties['scoringResultsDataSetId'])
    dataset_writer = DatasetWriter(client_context, dataset)
    dataset_writer.write(prediction, file_format='json')
    print("Data written successfully to platform")
